In [1]:
import torch
import torch.nn as nn
import json
from torch_geometric.loader import DataLoader
from pathlib import Path
import numpy as np
from tqdm import tqdm
from typing import Set
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict
import os

# Import your existing modules
from src.models import CoGraphNet
from src.data.document_dataset import DocumentGraphDataset
from src.train import FocalLoss


In [2]:
# -------------------------
# Create Plots Folder
# -------------------------
PLOT_DIR = "plots"
if not os.path.exists(PLOT_DIR):
    os.makedirs(PLOT_DIR)

In [3]:
# -------------------------
# Helper Functions for Calibration
# -------------------------
def compute_ece(probs, labels, n_bins=10):
    bins = np.linspace(0.0, 1.0, n_bins + 1)
    ece = 0.0
    max_probs = probs.max(axis=1)
    pred_labels = probs.argmax(axis=1)
    for i in range(n_bins):
        bin_lower = bins[i]
        bin_upper = bins[i+1]
        in_bin = (max_probs > bin_lower) & (max_probs <= bin_upper)
        prop_in_bin = np.mean(in_bin)
        if prop_in_bin > 0:
            accuracy_in_bin = np.mean(labels[in_bin] == pred_labels[in_bin])
            avg_confidence_in_bin = np.mean(max_probs[in_bin])
            ece += np.abs(avg_confidence_in_bin - accuracy_in_bin) * prop_in_bin
    return ece

In [4]:
def plot_reliability_diagram(probs, labels, n_bins=10, save_path=None):
    bins = np.linspace(0.0, 1.0, n_bins + 1)
    bin_centers = (bins[:-1] + bins[1:]) / 2.0
    max_probs = probs.max(axis=1)
    pred_labels = probs.argmax(axis=1)
    accuracies = []
    confidences = []
    for i in range(n_bins):
        bin_lower = bins[i]
        bin_upper = bins[i+1]
        in_bin = (max_probs > bin_lower) & (max_probs <= bin_upper)
        if np.sum(in_bin) > 0:
            accuracy = np.mean(labels[in_bin] == pred_labels[in_bin])
            confidence = np.mean(max_probs[in_bin])
        else:
            accuracy = 0.0
            confidence = 0.0
        accuracies.append(accuracy)
        confidences.append(confidence)
    plt.figure(figsize=(8, 6))
    plt.plot(bin_centers, accuracies, marker='o', label="Empirical Accuracy")
    plt.plot(bin_centers, confidences, marker='s', label="Average Confidence")
    plt.plot([0, 1], [0, 1], linestyle='--', color='gray', label="Perfect Calibration")
    plt.xlabel("Confidence")
    plt.ylabel("Accuracy")
    plt.title("Reliability Diagram")
    plt.legend()
    plt.grid(True)
    if save_path:
        plt.savefig(save_path)
    plt.close()

In [5]:
# -------------------------
# Utility Functions
# -------------------------
def get_all_categories(train_dir: str, test_dir: str) -> Set[str]:
    """Get all unique categories across all datasets."""
    categories = set()
    for data_dir in [train_dir, test_dir]:
        for file in Path(data_dir).glob('*.json'):
            with open(file, 'r', encoding='utf-8') as f:
                doc = json.load(f)
                if 'text' in doc and 'category' in doc and doc['text'].strip() and doc['category'].strip():
                    categories.add(doc['category'])
    return categories

In [6]:
def create_dataloaders(root: str, train_dir: str, test_dir: str, batch_size: int):
    """Create DataLoader instances with a train/validation split."""
    all_categories = get_all_categories(train_dir, test_dir)
    category_to_idx = {cat: idx for idx, cat in enumerate(sorted(all_categories))}
    num_classes = len(category_to_idx)
    
    full_train_dataset = DocumentGraphDataset(
        f"{root}/train", 
        train_dir, 
        category_to_idx=category_to_idx
    )
    
    dataset_size = len(full_train_dataset)
    val_size = int(dataset_size * 0.1)  # 10% for validation
    train_size = dataset_size - val_size
    
    generator = torch.Generator().manual_seed(42)
    train_dataset, val_dataset = torch.utils.data.random_split(
        full_train_dataset, 
        [train_size, val_size],
        generator=generator
    )
    
    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=4,
        pin_memory=True
    )
    
    val_loader = DataLoader(
        val_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=4,
        pin_memory=True
    )
    
    print("\nClass Distribution:")
    train_labels = [data.y.item() for data in train_dataset]
    val_labels = [data.y.item() for data in val_dataset]
    print("Training set:", {cls: train_labels.count(cls) for cls in set(train_labels)})
    print("Validation set:", {cls: val_labels.count(cls) for cls in set(val_labels)})
    
    return train_loader, val_loader, num_classes, category_to_idx

In [7]:
def validate(model, val_loader, criterion, device, category_to_idx, epoch, phase):
    """Validate the model and generate diagnostic plots including predicted vs true class counts."""
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    
    all_preds = []
    all_labels = []
    all_logits = []
    
    with torch.no_grad():
        for batch in val_loader:
            word_x = batch['word'].x.to(device)
            word_edge_index = batch['word', 'co_occurs', 'word'].edge_index.to(device)
            word_edge_weight = batch['word', 'co_occurs', 'word'].edge_attr.to(device)
            word_batch = batch['word'].batch.to(device)
            
            sent_x = batch['sentence'].x.to(device)
            sent_edge_index = batch['sentence', 'related_to', 'sentence'].edge_index.to(device)
            sent_edge_weight = batch['sentence', 'related_to', 'sentence'].edge_attr.to(device)
            sent_batch = batch['sentence'].batch.to(device)
            
            outputs = model(
                word_x, word_edge_index, word_batch, word_edge_weight,
                sent_x, sent_edge_index, sent_batch, sent_edge_weight
            )
            
            curr_logits = outputs[:batch.y.size(0)].cpu()
            all_logits.append(curr_logits)
            
            batch = batch.to(device)
            curr_batch_size = batch.y.size(0)
            loss = criterion(outputs[:curr_batch_size], batch.y)
            total_loss += loss.item() * curr_batch_size
            preds = outputs[:curr_batch_size].argmax(dim=1)
            correct += (preds == batch.y).sum().item()
            total += curr_batch_size
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(batch.y.cpu().numpy())
    
    # Concatenate logits and compute probabilities
    all_logits = torch.cat(all_logits, dim=0)
    all_probs = torch.softmax(all_logits, dim=1).numpy()
    all_labels_np = np.array(all_labels)
    
    # Plot histogram of predicted classes
    plt.figure(figsize=(8, 6))
    sns.histplot(all_preds, bins=np.arange(len(category_to_idx)+1)-0.5, kde=False)
    plt.xlabel("Predicted Class")
    plt.ylabel("Count")
    plt.title(f"Validation Prediction Distribution (Phase {phase}, Epoch {epoch+1})")
    plt.savefig(os.path.join(PLOT_DIR, f"val_pred_distribution_phase_{phase}_epoch_{epoch+1}.png"))
    plt.close()
    
    # NEW: Plot bar chart comparing predicted vs. true class counts
    indices = np.arange(len(category_to_idx))
    pred_counts = [np.sum(np.array(all_preds)==i) for i in range(len(category_to_idx))]
    true_counts = [np.sum(np.array(all_labels)==i) for i in range(len(category_to_idx))]
    width = 0.35
    plt.figure(figsize=(8,6))
    plt.bar(indices - width/2, pred_counts, width=width, label="Predicted")
    plt.bar(indices + width/2, true_counts, width=width, label="True")
    plt.xlabel("Class")
    plt.ylabel("Count")
    plt.title(f"Validation Class Distribution (Phase {phase}, Epoch {epoch+1})")
    plt.xticks(indices, list(range(len(category_to_idx))))
    plt.legend()
    plt.savefig(os.path.join(PLOT_DIR, f"val_class_distribution_phase_{phase}_epoch_{epoch+1}.png"))
    plt.close()
    
    # Calibration diagnostics
    logits_mean = all_logits.mean().item()
    logits_std = all_logits.std().item()
    print(f"[Phase {phase}, Epoch {epoch+1}] Logits Mean: {logits_mean:.4f}, Std: {logits_std:.4f}")
    ece = compute_ece(all_probs, all_labels_np, n_bins=10)
    print(f"[Phase {phase}, Epoch {epoch+1}] Expected Calibration Error (ECE): {ece:.4f}")
    plot_reliability_diagram(all_probs, all_labels_np, n_bins=10,
                             save_path=os.path.join(PLOT_DIR, f"reliability_diagram_phase_{phase}_epoch_{epoch+1}.png"))
    
    val_loss = total_loss / total
    val_acc = correct / total
    return val_loss, val_acc, all_preds, all_labels

In [8]:
def freeze_module(module):
    for param in module.parameters():
        param.requires_grad = False

def unfreeze_module(module):
    for param in module.parameters():
        param.requires_grad = True

In [9]:
def train_one_epoch(model, train_loader, optimizer, criterion, device, epoch, phase, num_classes):
    model.train()
    total_loss = 0
    batch_indices = []
    batch_unique_pred = []
    batch_unique_true = []
    
    for batch_idx, batch in enumerate(tqdm(train_loader, desc=f"Phase {phase} - Epoch {epoch+1}")):
        word_x = batch['word'].x.to(device)
        word_edge_index = batch['word', 'co_occurs', 'word'].edge_index.to(device)
        word_edge_weight = batch['word', 'co_occurs', 'word'].edge_attr.to(device)
        word_batch = batch['word'].batch.to(device)
        
        sent_x = batch['sentence'].x.to(device)
        sent_edge_index = batch['sentence', 'related_to', 'sentence'].edge_index.to(device)
        sent_edge_weight = batch['sentence', 'related_to', 'sentence'].edge_attr.to(device)
        sent_batch = batch['sentence'].batch.to(device)
        
        optimizer.zero_grad()
        outputs = model(
            word_x, word_edge_index, word_batch, word_edge_weight,
            sent_x, sent_edge_index, sent_batch, sent_edge_weight
        )
        
        batch = batch.to(device)
        curr_batch_size = batch.y.size(0)
        loss = criterion(outputs[:curr_batch_size], batch.y)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        
        preds = outputs[:curr_batch_size].argmax(dim=1)
        unique_pred = len(torch.unique(preds).cpu().numpy())
        unique_true = len(torch.unique(batch.y).cpu().numpy())
        batch_indices.append(batch_idx + 1)
        batch_unique_pred.append(unique_pred)
        batch_unique_true.append(unique_true)
    
    plt.figure(figsize=(10,6))
    plt.plot(batch_indices, batch_unique_pred, marker='o', linestyle='-', label='Unique Predicted Classes')
    plt.plot(batch_indices, batch_unique_true, marker='s', linestyle='-', label='Unique True Classes')
    plt.xlabel("Batch Index")
    plt.ylabel("Number of Unique Classes")
    plt.title(f"Training Unique Classes per Batch (Phase {phase}, Epoch {epoch+1})")
    plt.ylim(0, num_classes + 1)
    plt.legend()
    plt.grid(True)
    plt.savefig(os.path.join(PLOT_DIR, f"train_unique_classes_phase_{phase}_epoch_{epoch+1}.png"))
    plt.close()
    
    avg_loss = total_loss / len(train_loader)
    return avg_loss

In [10]:
# -------------------------
# Main Training Function with Phased Training
# -------------------------
def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # Use best hyperparameters:
    batch_size = 16
    hidden_dim = 148
    learning_rate = 0.000363022060695821
    total_phases = 4
    epochs_per_phase = 50  # 50 epochs per phase
    
    # Create dataloaders
    train_loader, val_loader, num_classes, category_to_idx = create_dataloaders(
        root="processed_graphs_ohsumed",
        train_dir="processed_data_ohsumed/train",
        test_dir="processed_data_ohsumed/test",
        batch_size=batch_size
    )

    dropout_rate = {
        'word': 0.3,
        'sent': 0.3,
        'fusion': 0.3,
        'co_graph': 0.3,
        'final': 0.3
    }

    dropout_config = {
        'word': True,
        'sent': True,
        'fusion': True,
        'co_graph': True,
        'final': True
    }
    
    # Create model with best params (note: num_layers set to 3, dropout_rate added)
    model = CoGraphNet(
        word_in_channels=768,
        sent_in_channels=768,
        hidden_channels=hidden_dim,
        num_word_layers=3,
        num_sent_layers=3,
        num_classes=num_classes,
        dropout_rate=dropout_rate,
        dropout_config=dropout_config
    ).to(device)
    
    # Compute separate class weights for training and validation sets
    train_labels = torch.tensor([data.y.item() for data in train_loader.dataset])
    train_class_counts = torch.bincount(train_labels, minlength=num_classes)
    train_total_samples = len(train_labels)
    train_class_weights = train_total_samples / (num_classes * train_class_counts.float())
    train_class_weights = train_class_weights.to(device)

    val_labels = torch.tensor([data.y.item() for data in val_loader.dataset])
    val_class_counts = torch.bincount(val_labels, minlength=num_classes)
    val_total_samples = len(val_labels)
    val_class_weights = val_total_samples / (num_classes * val_class_counts.float())
    val_class_weights = val_class_weights.to(device)
    
    # Create separate loss functions for training and validation
    train_criterion = FocalLoss(gamma=params['gamma'], weight=train_class_weights)
    val_criterion = FocalLoss(gamma=params['gamma'], weight=val_class_weights)
    
    overall_train_losses = defaultdict(list)
    overall_val_losses = defaultdict(list)
    overall_val_accs = defaultdict(list)
    
    best_val_acc = 0
    best_model_state = None
    
    # PHASE 1: Train Sentence Network Only
    print("\n--- Phase 1: Train Sentence Network Only ---")
    freeze_module(model.word_net)
    freeze_module(model.fusion)
    freeze_module(model.final_mlp)
    unfreeze_module(model.sent_net)
    
    optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()),
                                 lr=learning_rate, weight_decay=2.7910370279837748e-08)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=3, factor=0.1)
    
    for epoch in range(epochs_per_phase):
        train_loss = train_one_epoch(model, train_loader, optimizer, train_criterion, device, epoch, phase=1, num_classes=num_classes)
        val_loss, val_acc, all_preds, all_labels = validate(model, val_loader, val_criterion, device, category_to_idx, epoch, phase=1)
        scheduler.step(val_loss)
        overall_train_losses['Phase 1'].append(train_loss)
        overall_val_losses['Phase 1'].append(val_loss)
        overall_val_accs['Phase 1'].append(val_acc)
        print(f"Phase 1, Epoch {epoch+1}: Train Loss = {train_loss:.4f}, Val Loss = {val_loss:.4f}, Val Acc = {val_acc:.4f}")
    
    # PHASE 2: Train Word Network Only
    print("\n--- Phase 2: Train Word Network Only ---")
    freeze_module(model.sent_net)
    freeze_module(model.fusion)
    freeze_module(model.final_mlp)
    unfreeze_module(model.word_net)
    
    optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()),
                                 lr=learning_rate, weight_decay=2.7910370279837748e-08)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=3, factor=0.1)
    
    for epoch in range(epochs_per_phase):
        train_loss = train_one_epoch(model, train_loader, optimizer, train_criterion, device, epoch, phase=2, num_classes=num_classes)
        val_loss, val_acc, all_preds, all_labels = validate(model, val_loader, val_criterion, device, category_to_idx, epoch, phase=2)
        scheduler.step(val_loss)
        overall_train_losses['Phase 2'].append(train_loss)
        overall_val_losses['Phase 2'].append(val_loss)
        overall_val_accs['Phase 2'].append(val_acc)
        print(f"Phase 2, Epoch {epoch+1}: Train Loss = {train_loss:.4f}, Val Loss = {val_loss:.4f}, Val Acc = {val_acc:.4f}")
    
    # PHASE 3: Train Fusion Layer Only
    print("\n--- Phase 3: Train Fusion Layer Only ---")
    freeze_module(model.word_net)
    freeze_module(model.sent_net)
    freeze_module(model.final_mlp)
    unfreeze_module(model.fusion)
    
    optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()),
                                 lr=learning_rate, weight_decay=2.7910370279837748e-08)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=5, factor=0.5)
    
    for epoch in range(epochs_per_phase):
        train_loss = train_one_epoch(model, train_loader, optimizer, train_criterion, device, epoch, phase=3, num_classes=num_classes)
        val_loss, val_acc, all_preds, all_labels = validate(model, val_loader, val_criterion, device, category_to_idx, epoch, phase=3)
        scheduler.step(val_loss)
        overall_train_losses['Phase 3'].append(train_loss)
        overall_val_losses['Phase 3'].append(val_loss)
        overall_val_accs['Phase 3'].append(val_acc)
        print(f"Phase 3, Epoch {epoch+1}: Train Loss = {train_loss:.4f}, Val Loss = {val_loss:.4f}, Val Acc = {val_acc:.4f}")
    
    # PHASE 4: Fine-Tune Entire Model
    print("\n--- Phase 4: Fine Tune Entire Model ---")
    unfreeze_module(model.word_net)
    unfreeze_module(model.sent_net)
    unfreeze_module(model.fusion)
    unfreeze_module(model.final_mlp)
    
    optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()),
                                 lr=learning_rate, weight_decay=2.7910370279837748e-08)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=3, factor=0.1)
    
    for epoch in range(epochs_per_phase):
        train_loss = train_one_epoch(model, train_loader, optimizer, train_criterion, device, epoch, phase=4, num_classes=num_classes)
        val_loss, val_acc, all_preds, all_labels = validate(model, val_loader, val_criterion, device, category_to_idx, epoch, phase=4)
        scheduler.step(val_loss)
        overall_train_losses['Phase 4'].append(train_loss)
        overall_val_losses['Phase 4'].append(val_loss)
        overall_val_accs['Phase 4'].append(val_acc)
        print(f"Phase 4, Epoch {epoch+1}: Train Loss = {train_loss:.4f}, Val Loss = {val_loss:.4f}, Val Acc = {val_acc:.4f}")
        
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model_state = {
                'phase': 4,
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'val_acc': val_acc,
            }
            torch.save(best_model_state, os.path.join(PLOT_DIR, 'best_model.pt'))
    
    # Plot Overall Training and Validation Loss Across Phases
    phases = list(overall_train_losses.keys())
    plt.figure(figsize=(12, 5))
    for phase in phases:
        plt.plot(range(1, epochs_per_phase+1), overall_train_losses[phase], label=f'{phase} Train Loss')
        plt.plot(range(1, epochs_per_phase+1), overall_val_losses[phase], label=f'{phase} Val Loss')
    plt.xlabel("Epoch (per phase)")
    plt.ylabel("Loss")
    plt.title("Training vs. Validation Loss Across Phases")
    plt.legend()
    plt.grid(True)
    plt.savefig(os.path.join(PLOT_DIR, "overall_loss_across_phases.png"))
    plt.close()
    
    plt.figure(figsize=(12, 5))
    for phase in phases:
        plt.plot(range(1, epochs_per_phase+1), overall_val_accs[phase], label=f'{phase} Val Accuracy')
    plt.xlabel("Epoch (per phase)")
    plt.ylabel("Validation Accuracy")
    plt.title("Validation Accuracy Across Phases")
    plt.legend()
    plt.grid(True)
    plt.savefig(os.path.join(PLOT_DIR, "overall_val_accuracy_across_phases.png"))
    plt.close()
    
    print(f"Best Validation Accuracy: {best_val_acc:.4f}")

if __name__ == "__main__":
    main()


Loading existing valid indices from metadata
Found 10409 processed documents

Class Distribution:
Training set: {0: 379, 1: 144, 2: 58, 3: 1048, 4: 250, 5: 527, 6: 85, 7: 432, 8: 114, 9: 550, 10: 143, 11: 431, 12: 249, 13: 1127, 14: 198, 15: 172, 16: 265, 17: 358, 18: 171, 19: 469, 20: 499, 21: 79, 22: 1621}
Validation set: {0: 44, 1: 14, 2: 7, 3: 112, 4: 33, 5: 61, 6: 15, 7: 41, 8: 11, 9: 70, 10: 19, 11: 58, 12: 31, 13: 116, 14: 17, 15: 25, 16: 29, 17: 30, 18: 19, 19: 55, 20: 46, 21: 13, 22: 174}

--- Phase 1: Train Sentence Network Only ---


Phase 1 - Epoch 1: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 586/586 [05:06<00:00,  1.91it/s]


[Phase 1, Epoch 1] Logits Mean: 0.0133, Std: 0.1327
[Phase 1, Epoch 1] Expected Calibration Error (ECE): 0.0075
Phase 1, Epoch 1: Train Loss = 2.4678, Val Loss = 2.6417, Val Acc = 0.0490


Phase 1 - Epoch 2: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 586/586 [06:02<00:00,  1.62it/s]


[Phase 1, Epoch 2] Logits Mean: 0.0144, Std: 0.1115
[Phase 1, Epoch 2] Expected Calibration Error (ECE): 0.0104
Phase 1, Epoch 2: Train Loss = 2.4609, Val Loss = 2.6554, Val Acc = 0.0423


...

[Phase 4, Epoch 48] Logits Mean: -0.3726, Std: 0.9449
[Phase 4, Epoch 48] Expected Calibration Error (ECE): 0.0391
Phase 4, Epoch 48: Train Loss = 1.5880, Val Loss = 1.7894, Val Acc = 0.1808


Phase 4 - Epoch 49: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 586/586 [12:33<00:00,  1.29s/it]


[Phase 4, Epoch 49] Logits Mean: -0.3726, Std: 0.9449
[Phase 4, Epoch 49] Expected Calibration Error (ECE): 0.0391
Phase 4, Epoch 49: Train Loss = 1.5620, Val Loss = 1.7894, Val Acc = 0.1808


Phase 4 - Epoch 50: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 586/586 [07:25<00:00,  1.32it/s]


[Phase 4, Epoch 50] Logits Mean: -0.3726, Std: 0.9449
[Phase 4, Epoch 50] Expected Calibration Error (ECE): 0.0391
Phase 4, Epoch 50: Train Loss = 1.5736, Val Loss = 1.7895, Val Acc = 0.1808
Best Validation Accuracy: 0.1885


In [17]:
from sklearn.metrics import confusion_matrix

def create_test_loader(root: str, test_dir: str, batch_size: int, category_to_idx):
    """Create a DataLoader for the test dataset."""
    test_dataset = DocumentGraphDataset(
        f"{root}/test", 
        test_dir, 
        category_to_idx=category_to_idx
    )
    test_loader = DataLoader(
        test_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=4,
        pin_memory=True
    )
    return test_loader

def test_model(model, test_loader, device, category_to_idx):
    """Test the model on the test dataset, print accuracy, and plot the confusion matrix."""
    model.eval()
    total = 0
    correct = 0
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for batch in test_loader:
            word_x = batch['word'].x.to(device)
            word_edge_index = batch['word', 'co_occurs', 'word'].edge_index.to(device)
            word_edge_weight = batch['word', 'co_occurs', 'word'].edge_attr.to(device)
            word_batch = batch['word'].batch.to(device)
            
            sent_x = batch['sentence'].x.to(device)
            sent_edge_index = batch['sentence', 'related_to', 'sentence'].edge_index.to(device)
            sent_edge_weight = batch['sentence', 'related_to', 'sentence'].edge_attr.to(device)
            sent_batch = batch['sentence'].batch.to(device)
            
            outputs = model(
                word_x, word_edge_index, word_batch, word_edge_weight,
                sent_x, sent_edge_index, sent_batch, sent_edge_weight
            )
            curr_batch_size = batch.y.size(0)
            preds = outputs[:curr_batch_size].argmax(dim=1)
            correct += (preds == batch.y.to(device)).sum().item()
            total += curr_batch_size
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(batch.y.cpu().numpy())
    
    test_acc = correct / total
    print("Test Accuracy:", test_acc)
    
    # Plot confusion matrix
    cm = confusion_matrix(all_labels, all_preds)
    plt.figure(figsize=(8, 8))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
    plt.xlabel("Predicted Class")
    plt.ylabel("True Class")
    plt.title("Confusion Matrix on Test Data")
    plt.savefig(os.path.join(PLOT_DIR, "confusion_matrix_test.png"))
    plt.close()

# -------------------------
# Load the Test Dataset and Evaluate
# -------------------------
if __name__ == "__main__":
    # Create test loader using the same root and test_dir as in training
    root = "processed_graphs_ohsumed"
    test_dir = "processed_data_ohsumed/test"
    # Use best hyperparameters:
    batch_size = 16
    hidden_dim = 148
    learning_rate = 0.000363022060695821
    total_phases = 4
    epochs_per_phase = 50  # 50 epochs per phase
    device="cpu"
    # Re-use create_dataloaders to get the category mapping
    _, _, num_classes, category_to_idx = create_dataloaders(root, "processed_data_ohsumed/train", test_dir, batch_size)
    test_loader = create_test_loader(root, test_dir, batch_size, category_to_idx)
    
    # Load the best model state (assumed saved in the plots folder as "best_model.pt")
    best_state = torch.load(os.path.join(PLOT_DIR, "best_model.pt"), map_location=device)
    # Create model with best params (note: num_layers set to 3, dropout_rate added)
    model = CoGraphNet(
        word_in_channels=768,
        sent_in_channels=768,
        hidden_channels=hidden_dim,
        num_layers=3,
        num_classes=num_classes,
        dropout_rate=0.16716698278231815
    ).to(device)
    model.load_state_dict(best_state['model_state_dict'])
    
    # Evaluate the model on the test set and plot confusion matrix
    test_model(model, test_loader, device, category_to_idx)


Loading existing valid indices from metadata
Found 10409 processed documents

Class Distribution:
Training set: {0: 379, 1: 144, 2: 58, 3: 1048, 4: 250, 5: 527, 6: 85, 7: 432, 8: 114, 9: 550, 10: 143, 11: 431, 12: 249, 13: 1127, 14: 198, 15: 172, 16: 265, 17: 358, 18: 171, 19: 469, 20: 499, 21: 79, 22: 1621}
Validation set: {0: 44, 1: 14, 2: 7, 3: 112, 4: 33, 5: 61, 6: 15, 7: 41, 8: 11, 9: 70, 10: 19, 11: 58, 12: 31, 13: 116, 14: 17, 15: 25, 16: 29, 17: 30, 18: 19, 19: 55, 20: 46, 21: 13, 22: 174}
Loading existing valid indices from metadata
Found 12699 processed documents
Test Accuracy: 0.1785967399007796
